In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#Importing Data

df = pd.read_csv('../data/interim/radius3_selected_features.csv',index_col=[0])
X = df.drop(['Gini index','Country Name'],axis=1).to_numpy()

y = df['Gini index'].to_numpy()
groups = df['Country Name'].to_numpy()


#preprocessing pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

preprocess_steps = [('scaler',StandardScaler()),
                    ('ct',ColumnTransformer([('pca',PCA(n_components=2),list(range(14,20)))],remainder='passthrough'))]


#Setting up cross-validation
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=5)

#Setting up model
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

params = {'reg_lambda': 1.2000000000000002,
 'reg_alpha': 0.8,
 'n_estimators': 135,
 'max_depth': 3,
 'learning_rate': 0.4}

model = Pipeline(steps=preprocess_steps + [('xgb',XGBRegressor(**params))])


#Feature Names
columns = df.drop(['Country Name', 'Gini index'],axis=1).columns.to_list()
feature_names = [col for col in columns if col.find('Poverty') < 0]

feature_names = ['poverty_pca0','poverty_pca1'] + feature_names

In [ ]:
df.head()

In [ ]:
ct = ColumnTransformer([('pca',PCA(n_components=2),list(range(14,20)))],remainder='passthrough')

Xt = ct.fit_transform(X)

In [ ]:
ct.output_indices_